In [ ]:
from pathlib import Path

from typing import Iterable, Optional, Tuple, cast

import os

import json

import time

import numpy as np

from OSCC_postprocessing.cine.functions_videos import load_cine_video

from cinereader import cinereader


In [19]:
import cinereader
dir(cinereader)

['MetaData',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'cinereader',
 'dti',
 'njit',
 'np',
 'pd',
 'prange',
 'read',
 'read_image',
 'read_image_range',
 'read_metadata',
 'read_time_from_trigger',
 'read_timestamps',
 's',
 'unpacking_the_image']

In [5]:
def _iter_files(dir: Path) -> Iterable[Path]:
    return sorted(p for p in dir.iterdir() if p.is_file())


In [11]:
def _load_metadata(files: Iterable[Path]) -> tuple[int, float, tuple[float, float]]:
    env_cx = os.getenv("SCH_CENTRE_X")
    env_cy = os.getenv("SCH_CENTRE_Y")
    env_offset = os.getenv("SCH_OFFSET")
    if env_cx and env_cy and env_offset:
        plumes = int(os.getenv("SCH_PLUMES", "0"))
        centre = (float(env_cx), float(env_cy))
        offset = float(env_offset)
        return plumes, offset, centre

    for file in files:
        if file.suffix.lower() == ".json":
            with file.open("r", encoding="utf-8") as handle:
                data = json.load(handle)
                plumes = int(data["plumes"])
                offset = float(data["offset"])
                centre = (float(data["centre_x"]) + 4.0, float(data["centre_y"]) - 3.0)
                return plumes, offset, centre
    raise FileNotFoundError(
        
        "No metadata JSON found alongside the cine files.\n"
        "Provide SCH_CENTRE_X, SCH_CENTRE_Y, and SCH_OFFSET environment variables to run without metadata.\n"

        "Plese do the manual calibration in GUI.py first. "
    )

In [124]:

# Folder
import cupy as cp


Mie_dir = r"G:\MeOH_test\Mie"

Mie_dir_path = Path(Mie_dir)


if not Mie_dir_path.exists():
    raise FileNotFoundError(f"Data dir not found: {Mie_dir_path}")
else: 
    save_dir_Mie = Mie_dir_path / "Processed_Results"
    save_dir_Mie.mkdir(exist_ok=True)

    rotated_dir_Mie = save_dir_Mie / "Rotated_Videos"
    rotated_dir_Mie.mkdir(exist_ok=True)

    data_dir_Mie = save_dir_Mie / "Postprocessed_Data"
    data_dir_Mie.mkdir(exist_ok=True)

    files = list(_iter_files(Mie_dir_path))
    _, offset, centre = _load_metadata(files)

    cine_files = [f for f in files if f.suffix.lower() == ".cine"]
    if not cine_files:
        raise FileNotFoundError(f"No .cine files found in {Mie_dir_path}")

    for cine_file in cine_files:
        # Testing reading with PyCine multithreading
        print(cine_file)
        start_time = time.time()
        video = load_cine_video(str(cine_file)) # .astype(np.float16) / 4096.0
        video = cp.asarray(video).astype(cp.float16) *1.0/ 4096.0
        print(f"Time loding the video by Pycine: {time.time() - start_time:.3} s")
        
        # Testing cinereader
        start_time = time.time()
        _, cine_video, _ = cinereader.read(cine_file)
        cine_video = np.asarray(cine_video)
        cine_video = cp.asarray(cine_video).astype(cp.float16) / 4096.0
        print(f"Time loding the video by CineReader: {time.time() - start_time:.3} s")
        


G:\MeOH_test\Mie\T2_Mie Camera_1.cine
Video Info - Width: 768, Height: 768, Frames: 401
Time loding the video by Pycine: 0.23 s
Time loding the video by CineReader: 0.387 s


In [119]:
cine_video.dtype

dtype('float16')

In [95]:
import matplotlib.pyplot as plt
frame_num = 100
cine_video = np.asarray(cine_video)
offset = 0
plt.imshow(cine_video[frame_num, :, offset:].astype(np.float16)- np.flipud(video[frame_num, :, :-offset].astype(np.float16)))
# plt.imshow(cine_video[frame_num])
# plt.imshow(np.flipud(video[frame_num]))
plt.colorbar()

ValueError: operands could not be broadcast together with shapes (768,768) (768,0) 

In [84]:
np.asarray(cine_video)

array([[[ 82,  70,  96, ..., 103, 109,  96],
        [ 70,  70,  70, ...,  90,  96,  90],
        [ 77,  64,  70, ...,  96, 109,  90],
        ...,
        [ 97,  96, 109, ...,  97,  70,  97],
        [ 70,  96,  77, ...,  91,  97,  91],
        [ 90,  77,  77, ...,  70,  83,  97]],

       [[ 82,  70,  70, ..., 103,  96, 110],
        [ 83,  70,  70, ..., 102,  96,  90],
        [ 90,  77,  70, ...,  96, 109, 103],
        ...,
        [ 97,  96, 122, ...,  97,  84,  97],
        [ 96,  96, 115, ...,  77,  83,  91],
        [ 90,  77, 103, ...,  70,  70,  97]],

       [[ 82,  70,  70, ..., 103,  96,  96],
        [ 83,  83,  83, ...,  90,  96, 103],
        [ 64,  64,  70, ...,  83,  96,  77],
        ...,
        [ 83,  83,  83, ...,  70,  84,  97],
        [ 70,  96,  90, ...,  77,  83,  77],
        [102,  77,  90, ...,  83,  96,  97]],

       ...,

       [[ 82,  70,  83, ...,  90,  70, 110],
        [ 83,  83,  58, ...,  90,  83,  90],
        [ 90,  77,  70, ...,  96,  83,  90